In [163]:
import pandas as pd
import numpy as np
np.random.seed(0)
from copy import deepcopy
from sklearn.model_selection import train_test_split

import keras

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding, Bidirectional
from keras.optimizers import Adam

In [50]:
traindf=pd.read_csv('train.csv')
testdf=pd.read_csv('test.csv')

In [51]:
traindf

,Patient_ID,Family_Case_ID,Severity,Name,Birthday_year,Parents or siblings infected,Wife/Husband or children infected,Medical_Expenses_Family,Medical_Tent,City,Deceased
0,1,4696,3,Miss Linda Betty,NaN,0,0,225,NaN,Santa Fe,1
1,2,21436,1,Ms. Ramona Elvira,1966.0,0,1,1663,NaN,Albuquerque,0
2,3,7273,3,Mr. Mario Vernon,1982.0,0,0,221,NaN,Santa Fe,1
3,4,8226,3,Mr. Hector Joe,1997.0,0,0,220,NaN,Santa Fe,1
4,5,19689,3,Ms. Jennie Debra,1994.0,0,0,222,NaN,Santa Fe,0
...,...,...,...,...,...,...,...,...,...,...,...
895,896,6253,3,Ms. Linda Wilcox,1998.0,1,1,344,NaN,Santa Fe,0
896,897,6483,3,Mr. Haiden Vance,2006.0,0,0,258,NaN,Santa Fe,0
897,898,981,3,Miss Anaiya Love,1990.0,0,0,214,NaN,Taos,1
898,899,16418,2,Mr. Robert Williams,1994.0,1,1,812,NaN,Santa Fe,0


In [52]:
traindf.isna().sum()


Patient_ID                             0
Family_Case_ID                         0
Severity                               0
Name                                   0
Birthday_year                        177
Parents or siblings infected           0
Wife/Husband or children infected      0
Medical_Expenses_Family                0
Medical_Tent                         702
City                                   2
Deceased                               0
dtype: int64

In [63]:
traindf.mean()

Patient_ID                             450.856347
Family_Case_ID                       14286.119154
Severity                                 2.316258
Parents or siblings infected             0.380846
Wife/Husband or children infected        0.522272
Medical_Expenses_Family                892.749443
Deceased                                 0.615813
age                                     24.524990
dtype: float64

Drop Med tent column
drop 2 nan rows for city
fill birthday year with rnd norm 1990+-15

In [61]:
testdf.isna().sum()

AttributeError: 'DataFrame' object has no attribute 'range'

In [55]:
traindf['Birthday_year'].fillna(np.random.normal(1990,15),inplace=True)
testdf['Birthday_year'].fillna(np.random.normal(1990,15),inplace=True)
traindf.drop('Medical_Tent',axis=1,inplace=True)
testdf.drop('Medical_Tent',axis=1,inplace=True)
traindf.dropna(inplace=True)
testdf.dropna(inplace=True)

In [56]:
traindf['age']=2020-traindf['Birthday_year']
traindf.drop('Birthday_year',axis=1,inplace=True)
testdf['age']=2020-testdf['Birthday_year']
testdf.drop('Birthday_year',axis=1,inplace=True)

In [57]:
traindf

,Patient_ID,Family_Case_ID,Severity,Name,Parents or siblings infected,Wife/Husband or children infected,Medical_Expenses_Family,City,Deceased,age
0,1,4696,3,Miss Linda Betty,0,0,225,Santa Fe,1,3.539215
1,2,21436,1,Ms. Ramona Elvira,0,1,1663,Albuquerque,0,54.000000
2,3,7273,3,Mr. Mario Vernon,0,0,221,Santa Fe,1,38.000000
3,4,8226,3,Mr. Hector Joe,0,0,220,Santa Fe,1,23.000000
4,5,19689,3,Ms. Jennie Debra,0,0,222,Santa Fe,0,26.000000
...,...,...,...,...,...,...,...,...,...,...
895,896,6253,3,Ms. Linda Wilcox,1,1,344,Santa Fe,0,22.000000
896,897,6483,3,Mr. Haiden Vance,0,0,258,Santa Fe,0,14.000000
897,898,981,3,Miss Anaiya Love,0,0,214,Taos,1,30.000000
898,899,16418,2,Mr. Robert Williams,1,1,812,Santa Fe,0,26.000000


# NN'

In [254]:
nndf=deepcopy(traindf)
nndftest=deepcopy(testdf)

In [255]:
nndf.drop(['Patient_ID','Family_Case_ID','Name'],axis=1,inplace=True)
nndftest.drop(['Patient_ID','Family_Case_ID','Name'],axis=1,inplace=True)

In [256]:
nndf

,Severity,Parents or siblings infected,Wife/Husband or children infected,Medical_Expenses_Family,City,Deceased,age
0,3,0,0,225,Santa Fe,1,3.539215
1,1,0,1,1663,Albuquerque,0,54.000000
2,3,0,0,221,Santa Fe,1,38.000000
3,3,0,0,220,Santa Fe,1,23.000000
4,3,0,0,222,Santa Fe,0,26.000000
...,...,...,...,...,...,...,...
895,3,1,1,344,Santa Fe,0,22.000000
896,3,0,0,258,Santa Fe,0,14.000000
897,3,0,0,214,Taos,1,30.000000
898,2,1,1,812,Santa Fe,0,26.000000


In [257]:
nndf['Medical_Expenses_Family']/=1000
nndf['age']/=100
nndf['Severity']=(nndf['Severity']-1)/2

In [258]:
nndf.drop('age',axis=1,inplace=True)
nndftest.drop('age',axis=1,inplace=True)

In [259]:
nndf=pd.get_dummies(nndf,prefix=['City'])
nndftest=pd.get_dummies(nndftest,prefix=['City'])
features =nndf.drop(['Deceased'],axis=1)
labels =nndf['Deceased']

In [260]:
labels

0      1
1      0
2      1
3      1
4      0
      ..
895    0
896    0
897    1
898    0
899    0
Name: Deceased, Length: 898, dtype: int64

In [261]:
labels=pd.get_dummies(labels,prefix=['Deceased'])
labels=np.array(labels)
features=np.array(features)


In [262]:
X_train, X_val, y_train, y_val = train_test_split(features, 
                                                    labels, 
                                                    test_size=0.1, 
                                                    random_state=5, 
                                                    shuffle=True, 
                                                    stratify=labels
                                                   )

In [263]:
X_train

array([[0.5, 0. , 0. , ..., 0. , 1. , 0. ],
       [1. , 0. , 0. , ..., 1. , 0. , 0. ],
       [1. , 0. , 0. , ..., 0. , 1. , 0. ],
       ...,
       [1. , 0. , 0. , ..., 0. , 1. , 0. ],
       [0. , 0. , 0. , ..., 1. , 0. , 0. ],
       [1. , 0. , 0. , ..., 1. , 0. , 0. ]])

In [264]:
y_train

array([[1, 0],
       [0, 1],
       [0, 1],
       ...,
       [0, 1],
       [1, 0],
       [0, 1]], dtype=uint8)

In [265]:
features[0]

array([1.   , 0.   , 0.   , 0.225, 0.   , 1.   , 0.   ])

In [266]:
def build_model(input_size, output_size):
    model = Sequential()
    model.add(Dense(12, input_dim=input_size,activation='elu'))
    #model.add(Dropout(0.5))
    model.add(Dense(20,activation='elu'))
    model.add(Dense(20,activation='elu'))
    model.add(Dense(output_size, activation='hard_sigmoid') )
    # Compile the model
    model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [267]:

model = build_model(input_size=len(X_train[0]), output_size=len(y_train[0]))

model.fit(X_train, y_train, validation_data=(X_val,y_val), batch_size=1,epochs=100,verbose=1)



Train on 808 samples, validate on 90 samples
Epoch 1/100
808/808 [==============================] - 5s 7ms/step - loss: 0.6323 - accuracy: 0.6578 - val_loss: 0.6135 - val_accuracy: 0.6889
Epoch 2/100
808/808 [==============================] - 3s 4ms/step - loss: 0.5991 - accuracy: 0.6757 - val_loss: 0.6061 - val_accuracy: 0.6944
Epoch 3/100
808/808 [==============================] - 3s 4ms/step - loss: 0.6080 - accuracy: 0.6900 - val_loss: 0.6257 - val_accuracy: 0.6667
Epoch 4/100
808/808 [==============================] - 3s 4ms/step - loss: 0.5901 - accuracy: 0.6757 - val_loss: 0.5839 - val_accuracy: 0.7222
Epoch 5/100
808/808 [==============================] - 3s 4ms/step - loss: 0.5862 - accuracy: 0.6832 - val_loss: 0.5818 - val_accuracy: 0.7167
Epoch 6/100
808/808 [==============================] - 3s 4ms/step - loss: 0.5807 - accuracy: 0.6906 - val_loss: 0.5883 - val_accuracy: 0.7389
Epoch 7/100
808/808 [==============================] - 3s 4ms/step - loss: 0.5972 - accuracy: 0.6

KeyboardInterrupt: 